**Instructor**: Prof. Keith Chugg (chugg@usc.edu)

**Teaching Assistant**: Alexios Rustom (arustom@usc.edu)

**Book**: Watt, J., Borhani, R., & Katsaggelos, A. K. (2020). Machine learning refined: Foundations, algorithms, and applications. Cambridge University Press.

**Notebooks**: Written by Alexios Rustom (arustom@usc.edu) and Prof. Keith Chugg (chugg@usc.edu)

In [ ]:
import os
import numpy as np 
import matplotlib.pyplot as plt
from torchvision import datasets

In [ ]:
def load_MNIST_data(data_path, fashion=False, quiet=False):
    if not fashion:
        train_set = datasets.MNIST(data_path, download=True, train=True)
        test_set = datasets.MNIST(data_path, download=True, train=False)
    else:
        train_set = datasets.FashionMNIST(data_path, download=True, train=True)
        test_set = datasets.FashionMNIST(data_path, download=True, train=False)      
    x_train = train_set.data.numpy()
    y_train = train_set.targets.numpy()

    x_test = test_set.data.numpy()
    y_test = test_set.targets.numpy()
    
    N_train, H, W = x_train.shape
    N_test, H, W = x_test.shape

    if not quiet:
        print(f'The data are {H} x {W} grayscale images.')
        print(f'N_train = {N_train}')
        print(f'N_test = {N_test}')
    for i in set(y_train):
        N_i_train = np.sum(y_train==i)
        N_i_test = np.sum(y_test==i)
        if not quiet:
            print(f'Class {i}: has {N_i_train} train images ({100 * N_i_train / N_train : .2f} %), {N_i_test} test images ({100 * N_i_test/ N_test : .2f} %) ')
    return x_train, y_train, x_test, y_test

In [ ]:
USE_FASHION_MNIST = False
if USE_FASHION_MNIST:
    tag_name = 'FashionMNIST'
else:
    tag_name = 'MNIST'
    
x_train, y_train, x_test, y_test = load_MNIST_data('./data/', fashion=USE_FASHION_MNIST, quiet=False)

In [ ]:
plt.gray() # B/W Images
plt.figure(figsize = (10,9)) # Adjusting figure size
# Displaying a grid of 3x3 images
for i in range(9):
    index = np.random.randint(low=0, high=len(y_train), dtype=int)
    plt.subplot(3,3,i+1)
    plt.imshow(x_train[index])
plt.show()

In [ ]:
x_train_float = x_train.astype("float32") 
x_test_float = x_test.astype("float32")
# Normalization
x_train_normalized = x_train_float/255.0
x_test_normalized = x_test_float/255.0

In [ ]:
X_train = x_train_normalized.reshape(len(x_train_normalized),-1) #reshape to fed into PCA
X_test = x_test_normalized.reshape(len(x_test_normalized),-1) #reshape to fed into PCA

print(f'x_train_normalized.shape: {x_train_normalized.shape}')
print(f'X_train.shape: {X_train.shape}')

##  Principal Component Analysis

- The linear Autoencoder cost may have many minimizers, of which the set of principal components is a particularly important one. 


- The spanning set of principal components always provide a consistent skeleton for a dataset, with its members pointing in the dataset’s *largest directions of orthogonal variance*. 


- Employing this particular solution to the linear Autoencoder is often referred to as Principal Component Analysis, or PCA for short, in practice.

- This idea is illustrated for a prototypical $N = 2$ dimensional dataset in Figure 8.6 (book), where the general elliptical distribution of the data is shown in light grey. 


- A scaled version of the first principal component of this dataset points in the direction in which the dataset is most spread out, also called its largest direction of variance. 


- A scaled version of the second principal component points in the next most important direction in which the dataset is spread out that is orthogonal to the first.

- This special orthonormal minimizer of the linear Autoencoder is given by the eigenvectors of the so-called covariance matrix of the data.

- Denoting by $\mathbf{X}$ the $N \times P$ data matrix consisting of our $P$ mean-centered input points stacked column-wise

\begin{equation}
\mathbf{X} = 
\begin{bmatrix}
\vert  \,\,\,\,\,\,\, \vert  \,\,\,\,\,  \cdots  \,\,\,\,\, \vert \\
\,\, \mathbf{x}_1 \,\,\, \mathbf{x}_2 \,\,\, \cdots \,\,\, \mathbf{x}_P \\
\vert  \,\,\,\,\,\,\, \vert  \,\,\,\,\,  \cdots  \,\,\,\,\, \vert 
\end{bmatrix}
\end{equation}

- The orthogonal basis provided by this special solution (called the *Principal Components* of a dataset) can be computed (as a minimum of the Autoencoder cost function) as the *eigenvectors* of the corresponding *correlation matrix* of this data

\begin{equation}
\text{covariance matrix of } \, \mathbf{X}: = \, \frac{1}{P}\mathbf{X}^{\,} \mathbf{X}^T
\end{equation}

- Denoting the eigenvector/value decomposition of the covariance matrix 
$\frac{1}{P}\mathbf{X}^{\,} \mathbf{X}^T$ is given as

\begin{equation}
\frac{1}{P}\mathbf{X}^{\,} \mathbf{X}^T = \mathbf{V}^{\,}\mathbf{D}^{\,}\mathbf{V}^T
\end{equation}

- then above the orthonormal basis we recover is given precisely by the eigenvectors above, i.e., $\mathbf{C} = \mathbf{V}$.  


- Again, these are referred to in the jargon of machine learning as the *principal components* of the data.

- Moreover, the variance in each (principal component) direction is given precisely by the corresponding eigenvalue in $\mathbf{D}$.

### PCA for reduced dimensional representation

Now, let's apply PCA. 

In [ ]:
P = X_train.shape[0]
Kx = X_train.T @ X_train / P ## sample covariance matrix

e_vals, E_vecs = np.linalg.eig(Kx)

E_vecs = E_vecs.T
small_to_big = np.argsort(e_vals)
big_to_small = small_to_big[::-1]

e_vals = e_vals[big_to_small]
E_vecs = E_vecs[big_to_small]

Now that we have a basis of orthonormal eigenvectors of ${\bf X}^T {\bf X}$, we can express a given data vector

Let's see how the eigenvalues compare in value. 


 Also, the approximation is an orthogonal project, so 
\[
    \| {\bf x} - \hat{\bf x} \|^2 = \| \sum_{k=0}^{P-1} X_k |bf e_k 

In [ ]:
print(f'lambdas: {e_vals[:10]}')
plt.figure()
plt.stem(10 * np.log10(e_vals[:47]))
plt.grid(';')
plt.xlabel('eigen value index (max to min)')
plt.ylabel('eigen value in dB')

plt.figure()
plt.plot(10 * np.log10(e_vals))
plt.grid(';')
plt.xlabel('eigen value index (max to min)')
plt.ylabel('eigen value in dB')

H_W = len(e_vals)
percent_energy = np.zeros(H_W)
for k in range(H_W):
    percent_energy[k] = np.sum(e_vals[:k])
percent_energy = 100 * percent_energy / np.sum(e_vals)

print(percent_energy.shape)
plt.figure()
plt.plot(np.arange(H_W), percent_energy, color='b', label='energy')
plt.plot(np.arange(H_W), 100 - percent_energy, color='r', label='error energy')
plt.axhline(95, color='b', linestyle='--', label='95%')
plt.axhline(5, color='r', linestyle='--', label='5%')
plt.grid(';')
plt.legend()
plt.xlim([0,200])
plt.xlabel('eigen value index (max to min)')
plt.ylabel('Percent Energy')
plt.show()

In [ ]:
k_max = 50
coeff =  (E_vecs[:k_max] @ X_train.T).T
X_approx = coeff @ E_vecs[:k_max]
X_approx = X_approx.reshape(P, 28, 28)

In [ ]:
plt.gray() # B/W Images
plt.figure(figsize = (10,9))
# Displaying a grid of 3x3 images
for i in range(9):
    index = np.random.randint(low=0, high=len(y_train), dtype=int)
    plt.subplot(3,3,i+1)
    plt.imshow(X_approx[index])
    plt.title("Number:{}".format(y_train[index]),fontsize = 17)
plt.tight_layout()
plt.show()